# Assigment 2
### Juan Giraldo

## 1) Programming: Optimal Local and Global Pairwise Sequence Alignment

Given two DNA sequences, find in the best local and global alignment, its corresponding alignment score, the possibility of existing multiple optimal alignments, and, if the answer is positive, the set of all optimal alignments.

s(i,j) = {+2 if i and j match, -1 if mismatch}  
y(g) = -2g

First we load the data from the input file (The file needs to be in the same folder as the notebook for this to work) and store the values in variables

In [82]:
MATCH_SCORE = 2
MISMATCH_SCORE = -1
GAP = -2

In [83]:
f = open("2.in", "r")
x_seq = f.readline().strip()
y_seq = f.readline().strip()
print(x_seq)
print(y_seq)

GATTAC
GCATGCT


This method is used to print a matrix given an array of arrays

In [84]:
def print_matrix(matrix):
    formatted_matrix = ""
    for i in range(len(matrix)):
        row = ""
        for j in range(len(matrix[0])):
            row+= str(matrix[i][j])
            if j != (len(matrix[0])-1):
                row+=" "
        row+= "\n"
        formatted_matrix+=row
    print (formatted_matrix)

This method return a score depending if we have a match or a mismatch

In [85]:
def scoring(i, j):
    if match(i, j):
        return MATCH_SCORE
    else:
        return MISMATCH_SCORE

This method checks if there is a match in the sequences

In [86]:
def match(i, j):
    first_letter = y_seq[j-1]
    second_letter = x_seq[i-1]
    return first_letter == second_letter

## Global aligment

Now we will use the Needleman-Wunch algorithm to find the optimal global aligment, first we have to build the dynamic programing matrix

In [87]:
#Create the matrix using the two sequences
matrix =[[0 for x in range(len(y_seq)+1)] for y in range(len(x_seq)+1)]
#Fill the matrix using the algorithm
for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            #If we are in the first row we calculate the gap penalty for each space
            if i == 0:
                matrix[i][j] = j*GAP
            #If we are in the first column we calculate the gap penalty for each space
            elif j == 0:
                matrix[i][j] = i*GAP
            #If we are anywhere else we use the algorithm to calculate the values of the neighbors
            else:
                value1 = matrix[i-1][j-1] + scoring(i,j)
                value2 = matrix[i-1][j] + GAP
                value3 = matrix[i][j-1] + GAP
                #We choose the maximun value
                matrix[i][j] = max(value1, value2, value3)
print("Optimal score:\n"+str(matrix[i][j])+"\n")
print("Dynamic programming matrix:")
print_matrix(matrix)

Optimal score:
1

Dynamic programming matrix:
0 -2 -4 -6 -8 -10 -12 -14
-2 2 0 -2 -4 -6 -8 -10
-4 0 1 2 0 -2 -4 -6
-6 -2 -1 0 4 2 0 -2
-8 -4 -3 -2 2 3 1 2
-10 -6 -5 -1 0 1 2 0
-12 -8 -4 -3 -2 -1 3 1



Now we can proced with the traceback in order to find the optimal aligment score and the arrows

In [88]:
temp_seq_x=""
temp_seq_y=""
x_seq_index=len(x_seq)-1
y_seq_index=len(y_seq)-1
multiple_sol = False
#We are going to loop until we get to the top-left corner
while (i != 0 and j != 0):
    #We calculate the value of the neighbors and store their position
    value_hash = {}
    value1 = matrix[i-1][j-1]
    value_hash[value1] = [i-1,j-1]
    value2 = matrix[i-1][j]
    value_hash[value2] = [i-1,j]
    value3 = matrix[i][j-1] 
    value_hash[value3] = [i,j-1]
    #If we have a match in the sequences we move diagonally
    if(match(i,j)):
        i-=1
        j-=1
        #As we moved diagonally we add a base from x and y
        temp_seq_x=x_seq[x_seq_index]+temp_seq_x
        temp_seq_y=y_seq[y_seq_index]+temp_seq_y
        x_seq_index-=1
        y_seq_index-=1
        #We check if we have multiple aligments
        if (value1 <= value2) or (value1 <= value3):
            multiple_sol = True
    #If we don't have a match we use the values to find where to move
    else:
        if not multiple_sol:
            multiple_sol = len([value1, value2, value3]) != len(set([value1, value2, value3]))
        #We take the largest value of the neighbors and we move there
        positions = value_hash[max(value1, value2, value3)]
        #As we moved diagonally we add a base from x and y
        if((i-1 == positions[0]) and (j-1 == positions[1])):
            temp_seq_x=x_seq[x_seq_index]+temp_seq_x
            temp_seq_y=y_seq[y_seq_index]+temp_seq_y
            x_seq_index-=1
            y_seq_index-=1
        #As we moved left we add a base from y and a - to x
        elif((i == positions[0]) and (j-1 == positions[1])):
            temp_seq_y=y_seq[y_seq_index]+temp_seq_y
            temp_seq_x="-"+temp_seq_x
            y_seq_index-=1
        #As we moved up we add a base from x and a - to y
        elif((i-1 == positions[0]) and (j == positions[1])):
            temp_seq_x=x_seq[x_seq_index]+temp_seq_x
            temp_seq_y="-"+temp_seq_y
            x_seq_index-=1
        i = positions[0]
        j = positions[1]
print("Optimal aligment:")
print(temp_seq_x)
print(temp_seq_y)
print("\nMultiple aligments:\n"+("YES" if multiple_sol else "NO"))

Optimal aligment:
G-ATTAC-
GCATG-CT

Multiple aligments:
YES


## Local aligment

In [89]:
print(x_seq)
print(y_seq)

GATTAC
GCATGCT


This method checks if a value is greater than 0, if it isn't we change it to 0

In [90]:
def checkGreaterThanZero(value):
    return value if value > 0 else 0

Now we will use the Smith-Waterman algorithm to find the optimal global aligment, first we have to build the dynamic programing matrix

In [91]:
#We create a matrix using the sequences
matrix =[[0 for x in range(len(y_seq)+1)] for y in range(len(x_seq)+1)]
#We define an array to store our maximun value and its positions in the matrix
max_value = [0, 0, 0]
multiple = False
#We loop the matrix in order to fill it
for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            #If we are in the first row we add the gap penalty but we have to check if the value is greater than zero
            if i == 0:
                matrix[i][j] = checkGreaterThanZero(j*GAP)
            #If we are in the first column we add the gap penalty but we have to check if the value is greater than zero
            elif j == 0:
                matrix[i][j] = checkGreaterThanZero(i*GAP)
            #In any other place we use the algorithm to calculate the values checking that they are greater than zero
            #if not we turn them to zero
            else:
                value1 = checkGreaterThanZero(matrix[i-1][j-1] + scoring(i,j))
                value2 = checkGreaterThanZero(matrix[i-1][j] + GAP)
                value3 = checkGreaterThanZero(matrix[i][j-1] + GAP)
                #We select the largest value
                matrix[i][j] = max(value1, value2, value3)
                #Here we are assigning the max value and checkig if we have multiple aligments
                if matrix[i][j]>max_value[0]:
                    max_value[0] = matrix[i][j]
                    max_value[1] = i
                    max_value[2] = j
                    multiple = False
                elif matrix[i][j]==max_value[0]:
                    max_value[1] = i
                    max_value[2] = j
                    multiple = True
print("Optimal score:\n"+str(max_value[0])+"\n")
print("Dynamic programming matrix:")
print_matrix(matrix)

Optimal score:
4

Dynamic programming matrix:
0 0 0 0 0 0 0 0
0 2 0 0 0 2 0 0
0 0 1 2 0 0 1 0
0 0 0 0 4 2 0 3
0 0 0 0 2 3 1 2
0 0 0 2 0 1 2 0
0 0 2 0 1 0 3 1



Now we do the traceback from the largest value to a zero

In [92]:
#We start looping from the position of the largest value
i = max_value[1]
j = max_value[2]
aligment = ""
#We loop until we find a 0
while(matrix[i][j] != 0):
    #We add the base and then move diagonally
    aligment = str(x_seq[i-1]) + aligment
    i-=1
    j-=1
print("Optimal aligment:\n"+aligment)
print(aligment+"\n")
print("Multiple aligments:\n"+("YES" if multiple else "NO"))

Optimal aligment:
AT
AT

Multiple aligments:
NO


### f) Which parts of the program did you need to change in order to perform local rather than global pairwise sequence alignment? How many hours you put on this question?

I had to create a new method to convert any negative value to zero, and also had to change the way the backtracking was done. Instead of starting at the bottom right corner, I had to start at the largest value in the matrix and move diagonally until I hit a zero. It took me 5 hours to implement the two algorithms.

## 2) Similarity Search using BLAST

### a) Answer the following questions: 

#### 1. What is the number of hits reported by BLASTP?

The number of hits reported by BLASTP was 1000.

#### 2. What is the number of hits with a Total score of 160 or above?

The number of hits with a total score of 160 or above is 14.

#### 3. Which types of organisms do the proteins from the previous question belong to?

The proteins belong to betacoronavirus.

#### 4. Some of these organisms are parasitic. In which hosts are these organisms found?

The organisms are found in humans.

#### 5. What type of disorders do the viruses cause whose proteins got a score of 160 or above in this search?

They cause respiratory syndrome-related disorders.

#### 6. Give the alignment of the MERS envelope protein sequence with the SARS-CoV-2 coronavirus sequence. Specify the number of identical residues and % sequence identity, the number of similar residues and % similarity (in BLAST terminology: % positive), and the number and % of gaps.


Query 1   MLPFVQERIGLFIVNFFIFTVVCAITLLVCMAFLTATRLCVQCMTGFNTLLVQPALYLYN  60

          M  FV E  G  IVN  +  + C + LLV +A LTA RLC  C    N  LV+P+ Y+Y+

Sbjct 1   MYSFVSEETGTLIVNSVLLFLACVVFLLVTLAILTALRLCAYCCNIVNVSLVKPSFYVYS  60

Query  61  TGRSVYVKFQDSKPP  75

             ++       S+ P
             
Sbjct  61  RVKN----LNSSRVP  71

The number of identical residues is 7, the % of sequence identity is 37%, the number of similar residues is 39, the % similarity is 52% and the number and % of gaps is 4 and 5% respectively.



### b) Are the scores for the hits with max scores of 160 and above significantly affected when using the PAM70 scoring matrix instead of the BLOSUM62 matrix?

The scores are affected by only 2 points when using PAM70 instead of BLOSUM62, I would say that this change is not that significant.

### c) Does the alignment between MERS and SARS-CoV-2 proteins change when using the PAM70 scoring matrix instead of the BLOSUM62 matrix? If so, how?


The alignment changes a little bit, for example there are no gaps when using PAM70, also the identity percentage grows from 37% to 41% but the percentage of positives goes down from 52% to 51%. Overall the alignment is better when we use PAM70.

### d) What does the expect parameter mean? What will happen if the expect threshold is increased,from its default value of 10 to a 100?


The expect parameter means the expectation of finding the given sequence by random chance in the database. If we increase the value from 10 to 100 the significance threshold for reporting matches against the database will be larger and we will have a greater number of matches.


### e) Based on the respective alignments, would you say that the MERS envelope protein is very similar to the sequence for the SARS-CoV-2 envelope protein? Briefly justify your answer

I would say that they are not that similar because the identity percentage is below 50% and the positives are just a few points above 50%.

### f) MERS and SARS-CoV-2 both infect humans, but many of the envelope proteins with sequence similarity infect other mammals. One of the viruses with envelope protein sequence similarity infects the tapir. Which of these two envelope proteins infecting different species is more similar to the SARS-CoV-2 envelope protein: the MERS or the tapir virus?

I would say that the MERS is more similar to the SARS-CoV-2 envelope protein, because the identity percentage between these two is 37% and when trying to find the alignment for the tapir coronavirus and the SARS-CoV-2 envelope protein, BLAST did not show any results.


### g) Include how long you put on this question

Approximately 3 hours.

## 3) Multiple Sequence Alignment

### a) Use T-Coffee to obtain a multiple sequence alignment of these sequences. Report the multiple sequence alignment and the guide tree used for the alignment.

The alignment was:

   >pACYC184  
GAATTCCGGATGAGC----ATT-CATCAGGCGGGCAAGAATGTG--AA---TAAAGGCCG  
GATAAA---ACTTGTGCTTATTTTTCTTTACG-GTCT------TTAAAAAGGCCGTAATA  
TCCAGCTG-AACGGTCT---GGTTATAGGTACATTGAGCAACTGACTG-AAATGCCTCAA  
AATGTTCT---TTACGATGCCATTG---GGATATATCAACGGTG-GT-ATATC  

>pBR322  
TTCTCATGTTTGACAGCT-TAT-CATC-GATAAGCTTTAATGCGGTAG---TTTATCACA  
GTTAAAT-TGCTAACGCAGTCAG----GCACC-GTGT------ATGAA-ATCTAACAATG  
---CGCTC-ATCGTCATCCTCGGCACCGTCACCCTGGAT-GCTGTAGGCATAGGCTTGG  
TTATGCCGGTACTGCCGGGCCTCTTG---CGGGATATCGTCC-----A-TTCCG 

>pBR325  
AGGCCATGTTTGACAGCT-TAT-CATC-GATAAGCTTTAATGCGGTAG---TTTATCACA   
GTTAAAT-TGCTAACGCAGTCAG----GCACC-GTGT------ATGAA-ATCTAACAATG  
---CGCTC-ATCGTCATCCTCGGCACCGTCACCCTGGAT-GCTGTAGGCATAGGCTTGG  
TTATGCCGGTACTGCCGGGCCTCTTG---CGGGATATCGTCC-----A-TTCCG  

>pBR327  
TTCTCATGTTTGACAGCT-TAT-CATC-GATAAGCTTTAATGCGGTAG---TTTATCACA  
GTTAAAT-TGCTAACGCAGTCAG----GCACC-GTGT------ATGAA-ATCTAACAATG  
---CGCTC-ATCGTCATCCTCGGCACCGTCACCCTGGAT-GCTGTAGGCATAGGCTTGG  
TTATGCCGGTACTGCCGGGCCTCTTG---CGGGATATCGTCC-----A-TTCCG  

>pGEX2T  
ACGTTAT---CGACTGCACGGTGCACC-AA--TGCTTC-TGGCGTCAG---GCAGCCATC  
GGAAGCTGTGGTATGGCTGT-------GCAGG-TCGT------AAATCACTGC-ATAATT  
---CGT---GTCGCTCAAG-GCGCACTCCCGTTCTGGAT-AATGTTTTTTGCGCCGACAT  
CATAACGGTTCTGGCAAATATTCTGAAATGAGCTGTTGACAATTAATCATCGG  

>pHSG575  
TGATGTC---CGGCGGTGCTTT-TGCC-GTTACGCACCACCCCGTCAGTAGCTGAACAG  
-GAGGGAC-AGCTGATAGAAACAG----AAGCCACTGGAGCACCTCAAAAACACCATCA  
TA---CACTAAATCA-CTAAGTTGGCAGCATCACCC--GAC-GCACTTTGCGCCGAAT---A    
AATACCTGTGACGGAAGATCACTTC--GCAGAATAA---------AT-AAATC      

And the guide tree is:

((pACYC184:0.14250,((pBR322:0.00000,pBR327:0.00000):0.00500,pBR325:0.00500):0.14250):0.00250,pGEX2T:0.13250,pHSG575:0.13750);


### b) Obtain another multiple sequence alignment for the same sequences using the multiple sequence alignment program WebPRANK. Report the multiple sequence alignment and the guide tree used for constructing it.

The alignment was:

>pACYC184  
--------GAATTCCGGATG-------------AGCATTCATCAGGCGGGCAAGAATGTGAATAAAGGCCGGATAAAACTTGTGCTTATTTTTCTTTACGGTCTTTAAAAAGGCCGTAATAT--CC------------AGCTGAAC-----GGTCTGGTTATAGGTACATTGAGCAACTGAC---TGAA--------ATGCCTCAA-------------------------------------AA-------------------------------------------TGTTCTTTACG--------ATGCCATT---GGGATATATCAACGGTGG------------------TATATC----------

>pHSG575  
-------TGATGTCCGGCGG-------------TGCTTTT------------------------------------------------------------------------GCCGTTACGC--ACCACCCCGTCAGTAGCTGAACAGGAGGGACAGCTGATAGAAACA-----GAAGCCAC---TGGA--------GCACCTCAA-------------------------------------AAACACCATCATACACTAAATCACTAAGTTGGCAGCATCACCCGACGCACTTTGCGCCGAATAAATACCTGT---G----------ACGGAAGATCACTTCGCAGAATAAATAAATC----------

>pGEX2T  
-------ACGTTATCGACTGCACGGTGCACCAATGCTTCT------------------------------------------------------------------------GGCGTCAGGCAGCC------------ATCGGAAGCTGTGGTATGGCTGTGCAGGTCG-----TAAATCAC---TGCATAATTCGTGTCGCTCAAGGCGC-----------------ACTCCCGTTCTGGATAA-------------------------------------------TGTTTTTTGCGCCGACATCATAACGGTTCTGGCAAATATTCTGAAATGAGCTGTTGAC------AATTAATCATCGG-----

>pBR327  
----TTCTCATGTTTGACAGCTTATCATCGATAAGCTT------------------------------------------------------------------------------TAATGCGGTA------------GTTTATCACAGTTAAATTGCTAACGCAGTCA-----GGCACCGTGTATGAA--------ATCTAACAATGCGCTCATCGTCATCCTCGGCACCGTCACCCTGGATGC-------------------------------------------TGTAGGCATAGGCTTGGTTATGCCGGTACTGCC--------------GGGCCTCTTGC------GGGATATCGTCCATTCCG

>pBR325  
AGGC----CATGTTTGACAGCTTATCATCGATAAGCTT------------------------------------------------------------------------------TAATGCGGTA------------GTTTATCACAGTTAAATTGCTAACGCAGTCA-----GGCACCGTGTATGAA--------ATCTAACAATGCGCTCATCGTCATCCTCGGCACCGTCACCCTGGATGC-------------------------------------------TGTAGGCATAGGCTTGGTTATGCCGGTACTGCC--------------GGGCCTCTTGC------GGGATATCGTCCATTCCG

>pBR322  
----TTCTCATGTTTGACAGCTTATCATCGATAAGCTT------------------------------------------------------------------------------TAATGCGGTA------------GTTTATCACAGTTAAATTGCTAACGCAGTCA-----GGCACCGTGTATGAA--------ATCTAACAATGCGCTCATCGTCATCCTCGGCACCGTCACCCTGGATGC-------------------------------------------TGTAGGCATAGGCTTGGTTATGCCGGTACTGCC--------------GGGCCTCTTGC------GGGATATCGTCCATTCCG

And the guide tree is:

(((pBR322:0.0001,pBR327:0.0001):0.005,pBR325:0.005):0.14,((pGEX2T:0.1325,pHSG575:0.1375):0.0025,pACYC184:0.1425):0.0025);

### c) Recalculate the WebPRANK alignment using the guide tree produced by T-Coffee. The phylogenetic tree can be entered into the WebPRANK program in the Basic Alignment Options part of the form by using a string input. Report the resulting multiple sequence alignment and guide tree.

The alignment was:

>pBR322  
--------TTCTCAT----------GTTTGACAGCTTATCATCGATAAGCTTTAATGCGGTAGTTTATCACAGTTAAATTGCTAACGCAGTCAGGCACCGTGTATGAAATCTAACAATGCGCTCATCGTCATCCTCGGCACCGTCACC----CT--GGATGCTGTAGGCATA------GGCTTGGTTAT--------------------GCCGGT-----------------------------------------------------------------------ACTGCCGGGCCTCTTGCGGGAT-----------------------------ATCGTCCATTCCG----------------------------------------------------------

>pBR327  
--------TTCTCAT----------GTTTGACAGCTTATCATCGATAAGCTTTAATGCGGTAGTTTATCACAGTTAAATTGCTAACGCAGTCAGGCACCGTGTATGAAATCTAACAATGCGCTCATCGTCATCCTCGGCACCGTCACC----CT--GGATGCTGTAGGCATA------GGCTTGGTTAT--------------------GCCGGT-----------------------------------------------------------------------ACTGCCGGGCCTCTTGCGGGAT-----------------------------ATCGTCCATTCCG----------------------------------------------------------

>pBR325  
AGGC--------CAT----------GTTTGACAGCTTATCATCGATAAGCTTTAATGCGGTAGTTTATCACAGTTAAATTGCTAACGCAGTCAGGCACCGTGTATGAAATCTAACAATGCGCTCATCGTCATCCTCGGCACCGTCACC----CT--GGATGCTGTAGGCATA------GGCTTGGTTAT--------------------GCCGGT-----------------------------------------------------------------------ACTGCCGGGCCTCTTGCGGGAT-----------------------------ATCGTCCATTCCG----------------------------------------------------------
  
>pGEX2T  
----ACGT----TATCGACTGCACGGTGCACCAA---------------------------------------------TGCTTCTGGCGTCAGGC-------------------------------------------------AGC----CATCGGAAGCTG-------T------GGTATGGCTGT--------------------GCAGGTCG-----------------TAAA------------------------TCACTGCATAATTCGTGTCGCTCAAG-GCGCACTCCCGTTCTGGAT-----------------------------AATGTTTTTTGCGCCGAC-------------------ATCAT-----------------------------

>pHSG575  
-------T----GAT----------GTCCGGCGG---------------------------------------------TGCTTTTGCCGTTACGC-------------------------------------------------ACCACCCCGTCAGTAGCTG-------AACAGGAGGGACAGCTGATAGAAACAGAAGCCACTGGAGCACCTCAAAAACACCATCATACACTAAA------------------------TCACTAAGTTGGCAGCATCACCCGAC-GCAC---------------------------------------------------TTTGCGCCGAATAAATACCTGTGACGGAAGATCACTTCGCAGAATAAAT----------AAATC

>pACYC184  
------------GAA----------TTCCGGATG---------------------------------------------AGCATT---CATCAGGC-------------------------------------------------GGG----CA--AGAATGTG-------A------ATAAAGGCCGG---------------------------A-----------------TAAAACTTGTGCTTATTTTTCTTTACGGTCTTTAAAAAGGCCGTAATATCCAGCTGAACGGTCTGGTTATAGGTACATTGAGCAACTGACTGAAATGCCTCAAAATGTTCTTTACGATGCC-------------------AT-------TGGGATATATCAACGGTGGTATATC

And the guide tree is:

(((pACYC184:0.1311,pHSG575:0.1338):0.072,pGEX2T:0.1919):0.0543,((pBR327:0.0001,pBR325:0.0001):0.0001,pBR322:0.0001):0.2601);

### d) Comment on the differences between the multiple sequence alignments from (a), (b) and (c). Keep your answer as concise as possible.

The main difference between the three alignments is the number of gaps, for the alignment (a) we have a more optimal alignment, there are fewer gaps between the sequences and there is a low value of entropy, for the alignments (b) and (c) there is a very high number of gaps and a high value of entropy due to the mismatch of some base pairs in the sequences.    

### e) Include how long you put on this question.

Two hours aproximately